In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly 
import plotly.graph_objects as go
import plotly.express as px

from sklearn import linear_model
%matplotlib qt
pd.set_option('mode.chained_assignment', None)

# Load raw and save ready data

In [ ]:
# path = f'Experiments\MultiplyTemperature\Exp5(12.5)\Exp5_down.csv'

path = ''
while (path == '') or (not os.path.isfile(path)):
    path = input(f"Input data path: ")
_path_list = (path).split('\\')
folder_path = '\\'.join(_path_list[:-1])
data_name = _path_list[-1]
print(f'    Folder:\n{folder_path}\n    Data name:\n {data_name}')

In [ ]:
path

In [ ]:
df1 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up1.csv',
                  index_col=0)
# df2 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up2.csv',index_col=0)
# df3 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up3.csv',index_col=0)
# df4 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up4.csv',index_col=0)


In [ ]:
df1

In [ ]:
rules = {
    1: 'image_sweep_check',
    2: 'combine_check',  # 'inner_processor_check':'OK_inner'
    0: 'OK'
}
col_rules = {
    'Viscosity_mark': 'Viscosity_verbose',
    'Temperature_mark': 'Temperature_verbose',
}

df1 = df1.rename(columns=col_rules)
df1[['Viscosity_verbose','Temperature_verbose']] =\
    df1[['Viscosity_verbose','Temperature_verbose']].replace(rules)

# df2 = df2.rename(columns=col_rules)
# df2[['Viscosity_verbose','Temperature_verbose']] = df2[['Viscosity_verbose','Temperature_verbose']].replace(rules)
# df2['time'] =df2['time']+ df1['time'].max()+1

# df3 = df3.rename(columns=col_rules)
# df3[['Viscosity_verbose','Temperature_verbose']] = df3[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df3['time'] =df3['time']+ df2['time'].max()+1

# df4 = df4.rename(columns=col_rules)
# df4[['Viscosity_verbose','Temperature_verbose']] = df4[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df4['time'] =df4['time']+ df3['time'].max()+1
# df = pd.concat([df1,df2,df3,df4])

df = df1.dropna(subset=['Viscosity', 'Temperature'])
df

In [ ]:
np.any(df['time'].duplicated())

In [ ]:
assert not os.path.isfile(f'{folder_path}\AData_{data_name}'), 'Data already exists'
df.to_csv(f'{folder_path}\AData_{data_name}', index=0)


# Load

In [216]:
colors = plotly.colors.qualitative.Plotly
a = zip(
    ['a', 'b', 'c'],
    [round(i) for i in np.linspace(0, len(colors) - 1, len(['a', 'b', 'c']))],
)
{key: colors[color] for key, color in a}


{'a': '#636EFA', 'b': '#FFA15A', 'c': '#FECB52'}

In [ ]:
   # def get_values(self):
    #     y_lnD = np.log(nu)
    #     x_T = 1 / (T+273.15)

    #     nu0 = np.exp(w0)
    #     EkJ = 8.314 * w / 1000

    # go.Scatter(
    # x=self.data['time'],
    # y=self.data['Viscosity'],
    # name='Viscosity',
    # mode='markers',
    # marker=dict(size=5,
    #             color=self.data['Viscosity_verbose'].replace(colors_dict)))
    # go.Scatter(
    #     x=self.data['time'],
    #     y=self.data['Temperature'],
    #     name='Temperature',
    #     yaxis='y2',
    #     mode='markers',
    #     marker=dict(size=5,
    #                 symbol='square',
    #                 color=self.data['Temperature_verbose'].replace(colors_dict)),
    # )


In [237]:
class Data:
 
    data: pd.DataFrame = None

    def __init__(self, path=''):
        while (path == '') or (not os.path.isfile(path)):
            path = input(f"Input data path: ")
        _path_list = (path).split('\\')
        self.folder = '\\'.join(_path_list[:-1])
        self.name = _path_list[-1]
        print(f'Loaded: {self.name}')
        self._raw_data = pd.read_csv(path)

    def build_data(
            self,
            time_in_minutes=False,
            temperature=(13, 45),
            viscosity=(0, 400),
    ):
        self.conditions = dict(temperature=temperature, viscosity=viscosity)

        temperature_cond = ((temperature[0] < self._raw_data['Temperature']) &
                            (self._raw_data['Temperature'] < temperature[1]))
        viscosity_cond = ((viscosity[0] < self._raw_data['Viscosity']) &
                          (self._raw_data['Viscosity'] < viscosity[1]))

        self.data = self._raw_data[temperature_cond & viscosity_cond]
        if time_in_minutes: self.data['time'] / 60
        return self.data

    # def group(self):
    #     self.group_TV = self.data[['Temperature',
    #                                'Viscosity']].groupby(by='Temperature')

    def get_one_plot(self, name, y2_axix=False,symbol= 'circle',colors=plotly.colors.qualitative.Plotly):
        verboses = self.data[f'{name}_verbose'].unique()    
        rules = zip(
            verboses,
            [round(i) for i in np.linspace(0, len(colors) - 1, len(verboses))],
        )
        colors_rule={key: colors[color] for key, color in rules}
        

        scatter = go.Scatter(
            x=self.data['time'],
            y=self.data[name],
            name=name,
            mode='markers',
            yaxis='y2' if y2_axix else 'y',
            text=self.data[f'{name}_verbose'],
            hovertemplate='%{y}<br>%{text}',
            marker=dict(
                size=5,
                symbol=symbol,
                color=self.data[f'{name}_verbose'].replace(colors_rule),
            ),
        )
        return scatter

    def get_all_plots(self):
        # ,
        return [
            self.get_one_plot('Viscosity',colors=plotly.colors.sequential.Aggrnyl),
            self.get_one_plot('Temperature', y2_axix=True,symbol='star',colors=plotly.colors.sequential.Sunsetdark_r),
        ]

In [238]:
test = Data('Experiments\MultiplyTemperature\Exp4(10)\AData_Exp4_up.csv')
test.build_data()

Loaded: AData_Exp4_up.csv


,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
0,0.0,7.4,OK,14.9,OK
1,2.0,77.6,OK_inner,14.9,OK
2,4.0,7.5,OK,14.8,OK
3,6.0,78.0,OK,14.8,OK
4,8.0,77.7,OK_inner,14.8,OK
...,...,...,...,...,...
2315,4682.0,21.2,OK,40.0,OK_inner
2316,4684.0,21.0,OK,40.1,OK_inner
2317,4686.0,21.1,OK_inner,40.1,OK
2318,4688.0,21.1,OK_inner,40.1,OK


In [ ]:
# regression_done = False


# dfs = df[temperature_cond & viscosity_cond]

# temperature_cond2 = ((12 < df2['Temperature']) & (df2['Temperature'] < 45))
# viscosity_cond2 = ((0 < df2['Viscosity']) & (df2['Viscosity'] < 200))
# dfs2 = df2[temperature_cond2 & viscosity_cond2]
# dfs =df

In [ ]:
# plot_rules = {'processor_sweep':2, 'OK':0, 'error':4, 'value_combine':3,
#        'inner_processor_check':1}
# df[['Viscosity_verbose','Temperature_verbose']] = df[['Viscosity_verbose','Temperature_verbose']].replace(plot_rules)
# dfs['time'] = dfs['time'] / 60


In [ ]:
# T_goupT= dfs.groupby(by='Temperature')['Temperature']
# T_groupV=dfs.groupby(by='Temperature')['Viscosity']
# def foo(g):
#     whisker_width=0
#     q1 = g.quantile(0.45)
#     q3 = g.quantile(0.55)
#     iqr = q3 - q1
#     return (g >= q1 - whisker_width*iqr) & (g <= q3 + whisker_width*iqr)
#     # return np.abs((g - g.median()) / iqr) < 2.22
#     # return np.abs((g - g.mean()) / g.std(ddof=0)) < 1

# error_mask= T_groupV.apply(foo ).droplevel([0]).reset_index(drop=True).to_numpy()

# dfs = dfs[error_mask]

# # T_goupT2= dfs2.groupby(by='Temperature')['Temperature']
# # T_groupV2=dfs2.groupby(by='Temperature')['Viscosity']
# # dfs2
# dfs

In [ ]:
# dfs_median = dfs[['Temperature',
#                   'Viscosity']].groupby(by='Temperature').median()
# dfs_mean = dfs[['Temperature', 'Viscosity']].groupby(by='Temperature').mean()

# dfs_median2 = dfs2[['Temperature',
#                     'Viscosity']].groupby(by='Temperature').median()
# dfs_mean2 = dfs2[['Temperature', 'Viscosity']].groupby(by='Temperature').mean()


## Time plots

In [239]:
df =test.build_data()

fig, ax_v = plt.subplots()
ax_T = ax_v.twinx()

ax_v.scatter(df['time'], df['Viscosity'], color='red', marker='.')
ax_v.set_ylabel('Viscosity', color='red')

ax_T.scatter(df['time'], df['Temperature'], color='blue', marker='.')
ax_T.set_ylabel('Temperature', color='blue')


Text(0, 0.5, 'Temperature')

In [ ]:
scatters = test.get_all_plots()

fig = go.Figure()
fig.add_traces(scatters)
fig.update_layout(
    hovermode="x unified",
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    yaxis2=dict(
        overlaying='y',
        side='right',
        # range=[15, 20],
    ),
    # xaxis=dict(range=[0, 500], ),
)
fig.show()

## Temterature plots

In [ ]:
fig = go.Figure()
fig = px.scatter(
    dfs,
    x="Temperature",
    y="Viscosity",
    color='Viscosity_verbose',
)

fig.add_traces([
    go.Scatter(name='Median',
               x=dfs_median.index,
               y=dfs_median['Viscosity'],
               mode='lines',
               line=dict(color='black')),
    go.Scatter(name='Mean',
               x=dfs_mean.index,
               y=dfs_mean['Viscosity'],
               mode='lines',
               line=dict(color='red')),
])
# if len(dfs2):
#     fig.add_traces([
#         go.Scatter(name='Median2',
#                    x=dfs_median.index,
#                    y=dfs_median2['Viscosity'],
#                    mode='lines',
#                    line=dict(color='black', width=2, dash='dot')),
#         go.Scatter(name='Mean2',
#                    x=dfs_mean2.index,
#                    y=dfs_mean2['Viscosity'],
#                    mode='lines',
#                    line=dict(color='red', width=2, dash='dot')),
#     ])

if regression_done:
    x = np.array(dfs_median.index)
    y = nu0 * np.exp(EkJ * 1000 / 8.314 / (x+273.15))
    fig.add_trace(go.Scatter(name='Regression', x=x, y=y, mode='lines'))
fig.update_layout(hovermode="x unified", margin=dict(l=0, r=0, b=0, t=0, pad=0))
fig.show()

## OLS

In [ ]:
dfp = dfs_median
dfp

In [ ]:
T = np.array(dfp.index)
nu = np.array(dfp['Viscosity'])

In [ ]:
y_lnD = np.log(nu)
x_T = 1 / (T+273.15)



nu0 = np.exp(w0)
EkJ = 8.314 * w / 1000


X = np.array([x_T]).T
Y = y_lnD

In [ ]:
reg = linear_model.LinearRegression(fit_intercept=True)
reg.fit(X, Y)
w = reg.coef_[0]
w0 = reg.intercept_

In [ ]:
nu0 = np.exp(w0)
EkJ = 8.314 * w / 1000


In [ ]:
exp_name = folder_path.split('(')[-1][:-1]
print(
    f"Experiment= {data_name[6:-4]}: {exp_name}%",
    f'nu0=  {nu0: 6.3e}',
    f'E  =  {EkJ: 6.3f} kJ',
    sep='\n',
)
regression_done = True


## Saving regression

In [ ]:
dfp = dfs_mean

In [ ]:
assert not os.path.isfile(f'{folder_path}\BReg_{data_name[6:]}'), 'Data already exists'
dfp.to_csv(f'{folder_path}\BReg_{data_name[6:]}', index=0)